<a href="https://colab.research.google.com/github/Chandusatyasai/Chandusatyasai/blob/cognifyz/location_analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install folium geopandas --quiet

import pandas as pd
import folium
from folium.plugins import MarkerCluster
import random

random.seed(42)
data = {
    'Restaurant Name': [f'Restaurant {i}' for i in range(1, 101)],
    'Latitude': [random.uniform(28.40, 28.70) for _ in range(100)],
    'Longitude': [random.uniform(77.00, 77.30) for _ in range(100)],
    'City': ['Delhi'] * 100,
    'Locality': [random.choice(['Connaught Place', 'Karol Bagh', 'Saket', 'Dwarka', 'Rohini']) for _ in range(100)],
    'Rating': [round(random.uniform(2.5, 5.0), 1) for _ in range(100)],
    'Cuisine': [random.choice(['North Indian', 'Chinese', 'Italian', 'South Indian', 'Fast Food']) for _ in range(100)],
    'Price Range': [random.choice(['Low', 'Medium', 'High']) for _ in range(100)]
}

df = pd.DataFrame(data)


center_lat = df['Latitude'].mean()
center_lon = df['Longitude'].mean()
restaurant_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)
marker_cluster = MarkerCluster().add_to(restaurant_map)

for _, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['Restaurant Name']}<br>Cuisine: {row['Cuisine']}<br>Rating: {row['Rating']}",
        tooltip=row['Locality']
    ).add_to(marker_cluster)


restaurant_map


In [3]:

locality_stats = df.groupby('Locality').agg(
    Restaurant_Count=('Restaurant Name', 'count'),
    Average_Rating=('Rating', 'mean'),
    Most_Common_Cuisine=('Cuisine', lambda x: x.mode()[0]),
    Price_Level_Mode=('Price Range', lambda x: x.mode()[0])
).sort_values(by='Restaurant_Count', ascending=False)

print("📍 Locality-wise Restaurant Analysis:")
print(locality_stats)

top_area = locality_stats['Restaurant_Count'].idxmax()
top_rating = locality_stats['Average_Rating'].idxmax()

print("\n🔎 Insights:")
print(f"• Most restaurants are in: {top_area} ({locality_stats.loc[top_area, 'Restaurant_Count']} restaurants)")
print(f"• Highest average rating is in: {top_rating} ({locality_stats.loc[top_rating, 'Average_Rating']:.2f})")
print(f"• Most common cuisine in {top_area}: {locality_stats.loc[top_area, 'Most_Common_Cuisine']}")
print(f"• Price range mode in {top_area}: {locality_stats.loc[top_area, 'Price_Level_Mode']}")


📍 Locality-wise Restaurant Analysis:
                 Restaurant_Count  Average_Rating Most_Common_Cuisine  \
Locality                                                                
Rohini                         24        4.025000             Chinese   
Connaught Place                23        3.926087             Chinese   
Karol Bagh                     20        3.575000             Italian   
Saket                          17        3.747059        South Indian   
Dwarka                         16        3.868750             Chinese   

                Price_Level_Mode  
Locality                          
Rohini                    Medium  
Connaught Place              Low  
Karol Bagh                   Low  
Saket                     Medium  
Dwarka                       Low  

🔎 Insights:
• Most restaurants are in: Rohini (24 restaurants)
• Highest average rating is in: Rohini (4.02)
• Most common cuisine in Rohini: Chinese
• Price range mode in Rohini: Medium
